# 通过DCGAN实现人脸图像生成

作者:[ZMpursue](https://github.com/ZMpursue)  
日期:2020.10.26

本教程将通过一个示例对DCGAN进行介绍。在向其展示许多真实人脸照片（数据集：[Celeb-A Face](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)）后，我们将训练一个生成对抗网络（GAN）来产生新人脸。本文将对该实现进行详尽的解释，并阐明此模型的工作方式和原因。并不需要过多专业知识，但是可能需要新手花一些时间来理解的模型训练的实际情况。为了节省时间，请尽量选择GPU进行训练。


## 1 简介
本项目基于paddlepaddle，结合生成对抗网络（DCGAN）,通过弱监督学习的方式，训练生成真实人脸照片

### 1.1 什么是GAN？

生成对抗网络（Generative Adversarial Network [1]，简称GAN）是非监督式学习的一种方法，通过让两个神经网络相互博弈的方式进行学习。该方法最初由 lan·Goodfellow 等人于2014年提出，原论文见 [Generative Adversarial Network](https://arxiv.org/abs/1406.2661)。

  生成对抗网络由一个生成网络与一个判别网络组成。生成网络从潜在空间（latent space）中随机采样作为输入，其输出结果需要尽量模仿训练集中的真实样本。判别网络的输入为真实样本或生成网络的输出，其目的是将生成网络的输出从真实样本中尽可能分辨出来。而生成网络则要尽可能地欺骗判别网络。两个网络相互对抗、不断调整参数，其目的是将生成网络生成的样本和真实样本尽可能的区分开[2] ）。 
  
让$x$是代表图像的数据。$D(x)$是判别器网络，输出的概率为$x$来自训练数据还是生成器。在这里输入$D(x)$的$x$是CHW大小为3x128x128的图像。使得$x$来自训练数据时$D(x)$尽量接近1，$x$来自生成器时$D(x)$尽量接近0。$D(x)$也可以被认为是传统的二进制分类器。

对于生成器网络，$z$为从标准正态分布采样的潜在空间向量。$G(z)$表示生成器函数，该函数将矢量$z$映射到数据空间。生成器的目标是拟合训练数据($p_{data}$)的分布，以便可以从该估计分布中生成假样本($p_g$)。

所以，$D(G(z))$是生成器$G$输出是真实的图像的概率。如Goodfellow的论文所述，$D$和$G$玩一个minimax游戏，其中$D$尝试最大化其正确分类真假的可能性$logD(x)$，以及$G$试图最小化以下可能性$D$会预测其输出是假的$log(1-D(G(x)))$。

GAN的损失函数可表示为：

> $\underset{G}{\text{min}} \underset{D}{\text{max}}V(D,G) = \mathbb{E}_{x\sim p_{data}(x)}\big[logD(x)\big] + \mathbb{E}_{z\sim p_{z}(z)}\big[log(1-D(G(z)))\big]$

从理论上讲，此minimax游戏的解决方案是$p_g = p_{data}$，鉴别者会盲目猜测输入是真实的还是假的。但是，GAN的收敛理论仍在积极研究中，实际上GAN常常会遇到梯度消失/爆炸问题。  
生成对抗网络常用于生成以假乱真的图片。此外，该方法还被用于生成视频、三维物体模型等。


### 1.2 什么是DCGAN？

DCGAN是深层卷积网络与 GAN 的结合，其基本原理与 GAN 相同，只是将生成网络和判别网络用两个卷积网络（CNN）替代。为了提高生成样本的质量和网络的收敛速度，论文中的 DCGAN 在网络结构上进行了一些改进：

 * 取消 pooling 层：在网络中，所有的pooling层使用步幅卷积（strided convolutions）(判别器)和微步幅度卷积（fractional-strided convolutions）(生成器)进行替换。
 * 加入 batch normalization：在生成器和判别器中均加入batchnorm。
 * 使用全卷积网络：去掉了FC层，以实现更深的网络结构。
 * 激活函数：在生成器（G）中，最后一层使用Tanh函数，其余层采用 ReLu 函数 ; 判别器（D）中都采用LeakyReLu。  

  ### 1.3 本文的改进
         
   * 将Adam优化器beta1参数设置为0.8，具体请参考[原论文](https://arxiv.org/abs/1412.6980)
   * 将BatchNorm批归一化中momentum参数设置为0.5
   * 将判别器(D)激活函数由elu改为leaky_relu，并将alpha参数设置为0.2
   * 生成器输出，判别器输入改为[3,128,128]
   * 损失函数选用Softmax_with_cross_entropy
   ---
 

## 2 环境设置及数据集

环境：paddlepaddle、scikit-image、numpy、matplotlib  

在本教程中，我们将使用[Celeb-A Faces](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)数据集，该数据集可以在链接的网站或[AI Studio](https://aistudio.baidu.com/aistudio/datasetdetail/39207)中下载。数据集将下载为名为img_align_celeba.zip的文件。下载后，并将zip文件解压缩到该目录中。  
img_align_celeba目录结构应为： 
```
/path/to/project  
	-> img_align_celeba  
		-> 188242.jpg  
		-> 173822.jpg  
		-> 284702.jpg  
		-> 537394.jpg  
		...
```

### 2.3 数据集预处理
多线程处理，以裁切坐标(0,20)和(128,148)，将输入网络的图片裁切到128*128。

In [ ]:
from PIL import Image
import os.path
import os
import threading
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

'''多线程将图片缩放后再裁切到128*128分辨率'''
#裁切图片宽度
w = 128
#裁切图片高度
h = 128
#裁切点横坐标(以图片左上角为原点)
x = 0
#裁切点纵坐标
y = 20

def cutArray(l, num):
  avg = len(l) / float(num)
  o = []
  last = 0.0

  while last < len(l):
    o.append(l[int(last):int(last + avg)])
    last += avg

  return o
  
def convertjpg(jpgfile,outdir,width=w,height=h):
    img=Image.open(jpgfile)
    (l,h) = img.size
    rate = min(l,h) / width
    try:
        img = img.resize((int(l // rate),int(h // rate)),Image.BILINEAR)
        img = img.crop((x,y,width+x,height+y))
        img.save(os.path.join(outdir,os.path.basename(jpgfile)))
    except Exception as e:
        print(e)

class thread(threading.Thread):
    def __init__(self, threadID, inpath, outpath, files):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.inpath = inpath
        self.outpath = outpath
        self.files = files
    def run(self):
        count = 0
        try:
            for file in self.files:
                convertjpg(self.inpath + file,self.outpath)
                count = count + 1
        except Exception as e:
            print(e)
        print('已处理图片数量：' +  str(count))
            
if __name__ == '__main__':
    inpath = './work/img_align_celeba/'
    outpath = './work/imgs/'
    if not os.path.exists(outpath):
        os.mkdir(outpath)
    files =  os.listdir(inpath)
    files = cutArray(files,8)
    T1 = thread(1, inpath, outpath, files[0])
    T2 = thread(2, inpath, outpath, files[1])
    T3 = thread(3, inpath, outpath, files[2])
    T4 = thread(4, inpath, outpath, files[3])
    T5 = thread(5, inpath, outpath, files[4])
    T6 = thread(6, inpath, outpath, files[5])
    T7 = thread(7, inpath, outpath, files[6])
    T8 = thread(8, inpath, outpath, files[7])
    
    T1.start()
    T2.start()
    T3.start()
    T4.start()
    T5.start()
    T6.start()
    T7.start()
    T8.start()
    
    T1.join()
    T2.join()
    T3.join()
    T4.join()
    T5.join()
    T6.join()
    T7.join()
    T8.join()


## 3 模型组网
### 3.1 定义数据预处理工具-DataReader
具体参考[DataReader教程](https://www.paddlepaddle.org.cn/documentation/docs/zh/advanced_guide/data_preparing/static_mode/reader_cn.html)

In [ ]:
import os
import cv2
import numpy as np
import paddle.dataset as dataset
from skimage import io,color,transform
import matplotlib.pyplot as plt
import math
import time
import paddle
import paddle.fluid as fluid
import six

img_dim = 128

'''准备数据，定义Reader()'''
PATH = 'work/imgs/'
TEST = 'work/imgs/'
class DataGenerater:
    def __init__(self):
        '''初始化'''
        self.datalist = os.listdir(PATH)
        self.testlist = os.listdir(TEST)

    def load(self, image):
        '''读取图片'''
        img = io.imread(image)
        img = transform.resize(img,(img_dim,img_dim))
        img = img.transpose()
        img = img.astype('float32')
        return img

    def create_train_reader(self):
        '''给dataset定义reader'''

        def reader():
            for img in self.datalist:
                #print(img)
                try:
                    i = self.load(PATH + img)
                    yield i.astype('float32')
                except Exception as e:
                    print(e)
        return reader

    def create_test_reader(self,):
        '''给test定义reader'''
        def reader():
            for img in self.datalist:
                #print(img)
                try:
                    i = self.load(PATH + img)
                    yield i.astype('float32')
                except Exception as e:
                    print(e)
        return reader

def train(batch_sizes = 32):
    reader = DataGenerater().create_train_reader()
    return reader

def test():
    reader = DataGenerater().create_test_reader()
    return reader


### 3.2 测试DataReader并输出图片

In [ ]:
train_reader = paddle.batch(
    paddle.reader.shuffle(
        reader=train(), buf_size=128
    ),
    batch_size=128
)
for batch_id, data in enumerate(train_reader()):
    
    plt.figure(figsize=(15,15))
    try:
        for i in range(100):
            image = data[i].transpose()
            plt.subplot(10, 10, i + 1)
            plt.imshow(image, vmin=-1, vmax=1)
            plt.axis('off')
            plt.xticks([])
            plt.yticks([])
            plt.subplots_adjust(wspace=0.1, hspace=0.1)
        plt.suptitle('\n Training Images',fontsize=30)
        plt.show()
        break
    except IOError:
        print(IOError)

### 3.3 权重初始化
在 DCGAN 论文中，作者指定所有模型权重应从均值为0、标准差为0.02的正态分布中随机初始化。  
在Fluid中，调用fluid.initializer.NormalInitializer实现initialize设置

In [ ]:
conv_initializer=fluid.initializer.NormalInitializer(loc=0.0, scale=0.02)
bn_initializer=fluid.initializer.NormalInitializer(loc=1.0, scale=0.02)

### 3.4 定义网络功能模块
包括卷积层、转置卷积层、$BatchNorm$层、全连接层和卷积$BatchNorm$组 
* 转置卷积层：在生成器中，需要用随机采样值生成全尺寸图像，dcgan使用转置卷积层进行上采样，在Fluid中，调用 fluid.layers.conv2d_transpose 实现转置卷积。
* BatchNorm层：调用 fluid.layers.batch_norm 接口实现bn层，激活函数默认使用ReLu。
* 卷积层：调用 fluid.nets.simple_img_conv_pool 实现卷积池化组，卷积核大小为5x5，池化窗口大小为2x2，窗口滑动步长为2，激活函数类型由具体网络结构指定。



In [ ]:
use_cudnn = True
use_gpu = True
n = 0

###BatchNorm层
def bn(x, name=None, act=None,momentum=0.5):
    return fluid.layers.batch_norm(
        x,
        param_attr=fluid.ParamAttr(
            name=name+"_bn_weight_1_",
            initializer=bn_initializer,
            trainable=True),
        # 指定权重参数属性的对象
        bias_attr=None,
        # 指定偏置的属性的对象
        moving_mean_name=name + '3',
        # moving_mean的名称
        moving_variance_name=name + '4',
        # moving_variance的名称
        name=name,
        act=act,
        momentum=momentum,
    )


###卷积层
def conv(x, num_filters,name=None, act=None):
    return fluid.nets.simple_img_conv_pool(
        input=x,
        filter_size=5,
        num_filters=num_filters,
        pool_size=2,
        # 池化窗口大小
        pool_stride=2,
        # 池化滑动步长
        param_attr=fluid.ParamAttr(
            name=name+"_conv_weight_1_",
            initializer=conv_initializer,
            trainable=True),
        bias_attr=False,
        use_cudnn=use_cudnn,
        act=act
    )

###全连接层
def fc(x, num_filters, name=None, act=None):
    return fluid.layers.fc(
        input=x,
        size=num_filters,
        act=act,
        param_attr=name + 'w',
        bias_attr=name + 'b'
    )

###转置卷积层
def deconv(x, num_filters, name=None, filter_size=5, stride=2, dilation=1, padding=2, output_size=None, act=None):
    return fluid.layers.conv2d_transpose(
        input=x,
        param_attr=fluid.ParamAttr(
            name=name+"_conv_weight_2_",
            initializer=conv_initializer,
            trainable=True),
        bias_attr=False,
        num_filters=num_filters,
        # 滤波器数量
        output_size=output_size,
        # 输出图片大小
        filter_size=filter_size,
        # 滤波器大小
        stride=stride,
        # 步长
        dilation=dilation,
        # 膨胀比例大小
        padding=padding,
        use_cudnn=use_cudnn,
        # 是否使用cudnn内核
        act=act
        # 激活函数
    )

#卷积BatchNorm组
def conv_bn_layer(input,
                  ch_out,
                  filter_size,
                  stride,
                  padding,
                  act=None,
                  groups=64,
                  name=None):
    tmp = fluid.layers.conv2d(
        input=input,
        filter_size=filter_size,
        num_filters=ch_out,
        stride=stride,
        padding=padding,
        act=None,
        bias_attr=False,
        param_attr=fluid.ParamAttr(
            name=name+"_conv_weight_3_",
            initializer=conv_initializer,
            trainable=True),
    )
    return fluid.layers.batch_norm(
        input=tmp,
        act=act,
        param_attr=fluid.ParamAttr(
            name=name+"_bn_weight_2_",
            initializer=bn_initializer,
            trainable=True),
        # 指定权重参数属性的对象
        bias_attr=None,
        # 指定偏置的属性的对象
        moving_mean_name=name + '_bn_3',
        # moving_mean的名称
        moving_variance_name=name + '_bn_4',
        # moving_variance的名称
        name=name + '_bn_',
        momentum=0.5,
    )

def conv_layer(input,
                  ch_out,
                  filter_size,
                  stride,
                  padding,
                  act=None,
                  name=None):
    return fluid.layers.conv2d(
        input=input,
        filter_size=filter_size,
        num_filters=ch_out,
        stride=stride,
        padding=padding,
        act=None,
        bias_attr=False,
        param_attr=fluid.ParamAttr(
            name=name+"_conv_weight_4_",
            initializer=conv_initializer,
            trainable=True),
    )

### 3.5 判别器
如上文所述，生成器$D$是一个二进制分类网络，它以图像作为输入，输出图像是真实的（相对应$G$生成的假样本）的概率。输入$Shape$为[3,128,128]的RGB图像，通过一系列的$Conv2d$，$BatchNorm2d$和$LeakyReLU$层对其进行处理，然后通过全连接层输出的神经元个数为2，对应两个标签的预测概率。

* 将BatchNorm批归一化中momentum参数设置为0.5
* 将判别器(D)激活函数leaky_relu的alpha参数设置为0.2

> 输入:  为大小128x128的RGB三通道图片  
> 输出:  经过一层全连接层最后为shape为[batch_size,2]的Tensor

In [ ]:
###判别器
def D(x):
    # (128 + 2 * 1 - 4) / 2 + 1 = 64
    x = conv_layer(x, 128, 4, 2, 1, act=None, name='d_conv_1')
    x = fluid.layers.leaky_relu(x,alpha=0.2,name='d_leaky_relu_1')

    # (64 + 2 * 1 - 4) / 2 + 1 = 32
    x = conv_bn_layer(x, 256, 4, 2, 1, act=None, name='d_conv_bn_2')
    x = fluid.layers.leaky_relu(x,alpha=0.2,name='d_leaky_relu_2')

    # (32 + 2 * 1 - 4) / 2 + 1 = 16
    x = conv_bn_layer(x, 385, 4, 2, 1, act=None, name='d_conv_bn_3')
    x = fluid.layers.leaky_relu(x,alpha=0.2,name='d_leaky_relu_3')

    # (16 + 2 * 1 - 4) / 2 + 1 = 8
    x = conv_bn_layer(x, 768, 4, 2, 1, act=None, name='d_conv_bn_4')
    x = fluid.layers.leaky_relu(x,alpha=0.2,name='d_leaky_relu_4')

    # (8 + 2 * 1 - 4) / 2 + 1 = 4
    x = conv_bn_layer(x, 1024, 4, 2, 1, act=None, name='d_conv_bn_5')
    x = fluid.layers.leaky_relu(x,alpha=0.2,name='d_leaky_relu_5')

    # (4 + 2 * 1 - 4) / 2 + 1 = 2
    x = conv_bn_layer(x, 512, 4, 2, 1, act=None, name='d_conv_bn_6')
    x = fluid.layers.leaky_relu(x, alpha=0.2, name='d_leaky_relu_6')

    x = fluid.layers.reshape(x,shape=[-1, 2048])
    x = fc(x, 2, name='d_fc1')
    return x



### 3.6 生成器
生成器$G$旨在映射潜在空间矢量$z$到数据空间。由于我们的数据是图像，因此转换$z$到数据空间意味着最终创建具有与训练图像相同大小[3,128,128]的RGB图像。在网络设计中，这是通过一系列二维卷积转置层来完成的，每个层都与$BatchNorm$层和$ReLu$激活函数。生成器的输出通过$tanh$函数输出，以使其返回到输入数据范围[−1,1]。值得注意的是，在卷积转置层之后存在$BatchNorm$函数，因为这是DCGAN论文的关键改进。这些层有助于训练过程中的梯度更好地流动。  

**生成器网络结构**  
![](https://ai-studio-static-online.cdn.bcebos.com/ca0434dd681849338b1c0c46285616f72add01ab894b4e95848daecd5a72e3cb)

* 将$BatchNorm$批归一化中$momentum$参数设置为0.5

> 输入:Tensor的Shape为[batch_size,100]其中每个数值大小为0~1之间的float32随机数  
> 输出:3x128x128RGB三通道图片


In [ ]:
###生成器
def G(x):
    x = fluid.layers.reshape(x, shape=[-1, 100, 1, 1])

    # 2 * (1 - 1) - 2 * 0  + 4 = 4
    x = deconv(x, num_filters=2048, filter_size=4, stride=1, padding=0, name='g_deconv_0')
    x = bn(x, name='g_bn_1', act='relu', momentum=0.5)

    # 2 * (4 - 1) - 2 * 1  + 4 = 8
    x = deconv(x, num_filters=1024, filter_size=4, stride=2, padding=1, name='g_deconv_1')
    x = bn(x, name='g_bn_2', act='relu', momentum=0.5)

    # 2 * (8 - 1) - 2 * 1  + 4 = 16
    x = deconv(x, num_filters=1024 , filter_size=4, stride=2, padding=1, name='g_deconv_2')
    x = bn(x, name='g_bn_3', act='relu', momentum=0.5)

    # 2 * (16 - 1) - 2 * 1  + 4 = 32
    x = deconv(x, num_filters=768, filter_size=4, stride=2, padding=1, name='g_deconv_3')
    x = bn(x, name='g_bn_4', act='relu', momentum=0.5)

    # 1 * (16 - 1) - 2 * 1  + 3 = 32
    x = deconv(x, num_filters=512, filter_size=3, stride=1, padding=1, name='g_deconv_4')
    x = bn(x, name='g_bn_5', act='relu', momentum=0.5)

    # 2 * (32 - 1) - 2 * 1  + 4 = 64
    x = deconv(x, num_filters=512, filter_size=4, stride=2, padding=1, name='g_deconv_5')
    x = bn(x, name='g_bn_6', act='relu',momentum=0.5)

    # 2 * (64 - 1) - 2 * 1  + 4 = 128
    x = deconv(x, num_filters=3, filter_size=4, stride=2, padding=1, name='g_deconv_6', act='tanh')
    return x

### 3.7 损失函数
选用Softmax_with_cross_entropy,公式如下:

  $loss_j =  -\text{logits}_{label_j} +\log\left(\sum_{i=0}^{K}\exp(\text{logits}_i)\right), j = 1,..., K$
 

In [ ]:
###损失函数
def loss(x, label):
    return fluid.layers.mean(fluid.layers.softmax_with_cross_entropy(logits=x, label=label))

## 4 模型训练
 设置的超参数为：
 * 学习率：0.0002
 * 输入图片长和宽：128
 * Epoch: 8
 * Mini-Batch：128
 * 输入Tensor长度：100
 * Adam：Beta1：0.5，Beta2：0.999  
 
训练过程中的每一次迭代，生成器和判别器分别设置自己的迭代次数。为了避免判别器快速收敛到0，本教程默认每迭代一次，训练一次判别器，两次生成器。

In [ ]:
import IPython.display as display
import warnings
warnings.filterwarnings('ignore')

img_dim = 128
LEARENING_RATE = 0.0002
SHOWNUM = 12
epoch = 6
output = "work/Output/"
batch_size = 128
G_DIMENSION = 100
beta1=0.5
beta2=0.999
d_program = fluid.Program()
dg_program = fluid.Program()

###定义判别器program
# program_guard()接口配合with语句将with block中的算子和变量添加指定的全局主程序（main_program)和启动程序（start_progrom)
with fluid.program_guard(d_program):
    # 输入图片大小为128*128
    img = fluid.layers.data(name='img', shape=[None,3,img_dim,img_dim], dtype='float32')
    # 标签shape=1
    label = fluid.layers.data(name='label', shape=[None,1], dtype='int64')
    d_logit = D(img)
    d_loss = loss(x=d_logit, label=label)

###定义生成器program
with fluid.program_guard(dg_program):
    noise = fluid.layers.data(name='noise', shape=[None,G_DIMENSION], dtype='float32')
    # 噪声数据作为输入得到生成照片
    g_img = G(x=noise)
    g_program = dg_program.clone()
    g_program_test = dg_program.clone(for_test=True)

    # 判断生成图片为真实样本的概率
    dg_logit = D(g_img)

    # 计算生成图片被判别为真实样本的loss
    dg_loss = loss(
        x=dg_logit,
        label=fluid.layers.fill_constant_batch_size_like(input=noise, dtype='int64', shape=[-1,1], value=1)
    )

###优化函数
opt = fluid.optimizer.Adam(learning_rate=LEARENING_RATE,beta1=beta1,beta2=beta2)
opt.minimize(loss=d_loss)
parameters = [p.name for p in g_program.global_block().all_parameters()]
opt.minimize(loss=dg_loss, parameter_list=parameters)

train_reader = paddle.batch(
    paddle.reader.shuffle(
        reader=train(), buf_size=110000
    ),
    batch_size=batch_size
)
test_reader = paddle.batch(
    paddle.reader.shuffle(
        reader=test(), buf_size=10000
    ),
    batch_size=10
)
###执行器
if use_gpu:
    exe = fluid.Executor(fluid.CUDAPlace(0))
else:
    exe = fluid.Executor(fluid.CPUPlace())
start_program = fluid.default_startup_program()
exe.run(start_program)
#加载模型
# fluid.io.load_persistables(exe,'work/Model/D/',d_program)
# fluid.io.load_persistables(exe,'work/Model/G/',dg_program)

###训练过程
t_time = 0
losses = [[], []]
# 判别器迭代次数
NUM_TRAIN_TIME_OF_DG = 2
# 最终生成的噪声数据
const_n = np.random.uniform(
    low=0.0, high=1.0,
    size=[batch_size, G_DIMENSION]).astype('float32')
test_const_n = np.random.uniform(
    low=0.0, high=1.0,
    size=[100, G_DIMENSION]).astype('float32')

#plt.ion()
now = 0
for pass_id in range(epoch):
    fluid.io.save_persistables(exe, 'work/Model/G', dg_program)
    fluid.io.save_persistables(exe, 'work/Model/D', d_program)
    # enumerate()函数将一个可遍历的数据对象组合成一个序列列表
    for batch_id, data in enumerate(train_reader()):  
        if len(data) != batch_size:
            continue

        # 生成训练过程的噪声数据
        noise_data = np.random.uniform(
            low=0.0, high=1.0,
            size=[batch_size, G_DIMENSION]).astype('float32')
        # 真实图片
        real_image = np.array(data)
        # 真实标签
        real_labels = np.ones(shape=[batch_size,1], dtype='int64')
        # 虚假标签
        fake_labels = np.zeros(shape=[batch_size,1], dtype='int64')
        s_time = time.time()
        # 虚假图片
        generated_image = exe.run(g_program,
                                  feed={'noise': noise_data},
                                  fetch_list=[g_img])[0]


        ###训练判别器
        # D函数判断虚假图片为假的loss
        d_loss_1 = exe.run(d_program,
                           feed={
                               'img': generated_image,
                               'label': fake_labels,
                           },
                           fetch_list=[d_loss])[0][0]
        # D函数判断真实图片为真的loss
        d_loss_2 = exe.run(d_program,
                           feed={
                               'img': real_image,
                               'label': real_labels,
                           },
                           fetch_list=[d_loss])[0][0]

        d_loss_n = d_loss_1 + d_loss_2
        losses[0].append(d_loss_n)

        ###训练生成器
        for _ in six.moves.xrange(NUM_TRAIN_TIME_OF_DG):
            # uniform()方法从一个均匀分布[low,high)中随机采样
            noise_data = np.random.uniform(  
                low=0.0, high=1.0,
                size=[batch_size, G_DIMENSION]).astype('float32')
            dg_loss_n = exe.run(dg_program,
                                feed={'noise': noise_data},
                                fetch_list=[dg_loss])[0][0]
        losses[1].append(dg_loss_n)
        t_time += (time.time() - s_time)
        if batch_id % 100 == 0:
            if not os.path.exists(output):
                os.makedirs(output)
            # 每轮的生成结果
            generated_image = exe.run(g_program_test, feed={'noise': test_const_n}, fetch_list=[g_img])[0]
            imgs = []
            plt.figure(figsize=(15,15))
            try:
                for i in range(100):
                    image = generated_image[i].transpose()
                    image = np.where(image > 0, image, 0)
                    plt.subplot(10, 10, i + 1)
                    plt.imshow(image, vmin=-1, vmax=1)
                    plt.axis('off')
                    plt.xticks([])
                    plt.yticks([])
                    plt.subplots_adjust(wspace=0.1, hspace=0.1)
                msg = 'Epoch ID={0} Batch ID={1} \n\n D-Loss={2} G-Loss={3}'.format(pass_id, batch_id, d_loss_n, dg_loss_n)
                plt.suptitle(msg,fontsize=20)
                plt.draw()
                plt.savefig('{}/{:04d}_{:04d}.png'.format(output, pass_id, batch_id),bbox_inches='tight')
                plt.pause(0.01)
                display.clear_output(wait=True)
            except IOError:
                print(IOError)


#plt.ioff()
plt.close()


In [ ]:
plt.figure(figsize=(15, 6))
x = np.arange(len(losses[0]))
plt.title('Generator and Discriminator Loss During Training')
plt.xlabel('Number of Batch')
plt.plot(x,np.array(losses[0]),label='D Loss')
plt.plot(x,np.array(losses[1]),label='G Loss')
plt.legend()
plt.savefig('work/Generator and Discriminator Loss During Training.png')
plt.show()

## 5 模型评估
### 生成器$G$和判别器$D$的损失与迭代变化图
![](https://ai-studio-static-online.cdn.bcebos.com/8c94b567738c423ba5a421a40ccf5f57fd8defc3b62d46f28cc5d3aa3439bf42)

### 对比真实人脸图像（图一）和生成人脸图像（图二）
#### 图一
![](https://ai-studio-static-online.cdn.bcebos.com/622fff7b67a240ff8a1fceb209fc27445c929099365c44e8bd006569bf26e0a6)
### 图二
![](https://ai-studio-static-online.cdn.bcebos.com/ed33cf82762d4ae79feef82b77604273303cc38b8f5d4728a632b492f2665ed4)


## 6 模型预测
### 输入随机数让生成器$G$生成随机人脸

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import IPython.display as display

'''定义超参数'''
img_dim = 128
output = "Output/"
G_DIMENSION = 100
#生成图片数量
number = 20
use_gpu = True
dg_program = fluid.Program()
d_program = fluid.Program()


###定义判别器program
# program_guard()接口配合with语句将with block中的算子和变量添加指定的全局主程序（main_program)和启动程序（start_progrom)
with fluid.program_guard(d_program):
    # 输入图片大小为128*128
    img = fluid.layers.data(name='img', shape=[None,3,img_dim,img_dim], dtype='float32')
    # 标签shape=1
    label = fluid.layers.data(name='label', shape=[None,1], dtype='int64')
    d_logit = D(img)
    d_loss = loss(x=d_logit, label=label)

###定义生成器program
with fluid.program_guard(dg_program):
    noise = fluid.layers.data(name='noise', shape=[None,G_DIMENSION], dtype='float32')
    # 噪声数据作为输入得到生成照片
    g_img = G(x=noise)
    g_program = dg_program.clone()
    g_program_test = dg_program.clone(for_test=True)

    # 判断生成图片为真实样本的概率
    dg_logit = D(g_img)

    # 计算生成图片被判别为真实样本的loss
    dg_loss = loss(
        x=dg_logit,
        label=fluid.layers.fill_constant_batch_size_like(input=noise, dtype='int64', shape=[-1,1], value=1)
    )

if use_gpu:
    exe = fluid.Executor(fluid.CUDAPlace(0))
else:
    exe = fluid.Executor(fluid.CPUPlace())
start_program = fluid.default_startup_program()
exe.run(start_program)
fluid.io.load_persistables(exe,'work/Model/D/',d_program)
fluid.io.load_persistables(exe,'work/Model/G/',dg_program)
try:
    const_n = []
    for m in range(number):
        noise = np.random.uniform(low=0.0, high=1.0,size=[G_DIMENSION]).astype('float32')
        const_n.append(noise)
    const_n = np.array(const_n).astype('float32')
    generated_image = exe.run(g_program, feed={'noise': const_n}, fetch_list=[g_img])[0]
    for j in range(number):
        image = generated_image[j].transpose()
        plt.figure(figsize=(4,4))
        plt.imshow(image)
        plt.axis('off')
        plt.xticks([])
        plt.yticks([])
        plt.subplots_adjust(wspace=0.1, hspace=0.1)
        plt.savefig('work/Generate/generated_' + str(j + 1), bbox_inches='tight')
        plt.close()
except IOError:
    print(IOError)

## 8 项目总结
简单介绍了一下DCGAN的原理，通过对原项目的改进和优化，一步一步依次对生成器和判别器以及训练过程进行介绍。
DCGAN采用一个随机噪声向量作为输入，输入通过与CNN类似但是相反的结构，将输入放大成二维数据。采用这种结构的生成模型和CNN结构的判别模型，DCGAN在图片生成上可以达到相当可观的效果。本案例中，我们利用DCGAN生成了人脸照片，您可以尝试更换数据集生成符合个人需求的图片，或尝试修改网络结构观察不一样的生成效果。

## 9 参考文献
[1] Goodfellow, Ian J.; Pouget-Abadie, Jean; Mirza, Mehdi; Xu, Bing; Warde-Farley, David; Ozair, Sherjil; Courville, Aaron; Bengio, Yoshua. Generative Adversarial Networks. 2014. arXiv:1406.2661 [stat.ML].

[2] Andrej Karpathy, Pieter Abbeel, Greg Brockman, Peter Chen, Vicki Cheung, Rocky Duan, Ian Goodfellow, Durk Kingma, Jonathan Ho, Rein Houthooft, Tim Salimans, John Schulman, Ilya Sutskever, And Wojciech Zaremba, Generative Models, OpenAI, [April 7, 2016]

[3] alimans, Tim; Goodfellow, Ian; Zaremba, Wojciech; Cheung, Vicki; Radford, Alec; Chen, Xi. Improved Techniques for Training GANs. 2016. arXiv:1606.03498 [cs.LG].

[4] Radford A, Metz L, Chintala S. Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks[J]. Computer Science, 2015.

[5]Kingma D , Ba J . Adam: A Method for Stochastic Optimization[J]. Computer ence, 2014.